# Qubit Tapering 
### in the Stabilizer Subspace Projection formalism
Here, we take a look at the qubit reduction technique of [tapering](https://arxiv.org/abs/1701.08213) and an implementation based on the core `S3_projection` class. Unlike [Contextual-Subspace VQE](https://doi.org/10.22331/q-2021-05-14-456), this technique is *exact*, in the sense that it perfectly preserves the energy spectrum of the input operator.

At the core of qubit tapering is a symmetry of the Hamiltonian, which in this case means a set of universally commuting operators. The idea is that these operators must be simultaneously measureable and so can be treated independently of the remaining Hamiltonian terms. The method works by finding an independent generating set for the symmetry and seeks to find the 'correct' assignment of eigenvalues (called a *sector*), which completely determines the measurement outcome of the symmetry operators. Once this is obtained, the theory of stabilizers allows us to rotate the symmetry generators onto single Pauli $X$ operators, and since they must commute universally every operator of the rotated Hamiltonian will consist of an identity or Pauli $X$ in the corresponding qubit position. This means we can drop the qubit from the Hamiltonian, leaving in its place the eigenvalue determined by the chosen sector.

In [1]:
import numpy as np
import openfermion as of
import openfermionpyscf as ofpyscf
import qreduce.utils.qonversion_tools as qonvert
from qreduce.tapering import tapering
from qreduce.utils.operator_toolkit import exact_gs_energy, plot_ground_state_amplitudes

First, we shall construct a molecule using OpenFermion with PySCF the underlying quantum chemistry package. The resulting fermionic operator will be mapped onto qubits via the Jordan-Wigner transformation.

In [18]:
# Set molecule parameters

# geometry=[
#     ('H',(0.2774,0.8929,0.2544)),
#     ('O',(0,0,0)),
#     ('H',(0.6068,-0.2383,-7.7169))
#     ]

geometry=[
    ('H',(0.0, 0.0, 0.0)),
    ('H',(0.0, 0.74, 0.0)),
    ]


#geometry=[
#    ('H', (0.0,0.0,0.0)),
#    ('H', (2.45366053071732,0.0,0.0)),
#    ('H', (2.45366053071732,2.45366053071732,0.0)),
#    ('H', (0.0,2.45366053071732,0.0))
#     ]    
    
basis = 'sto-3g'
multiplicity = 1
charge = 0

molecule_data = of.MolecularData(geometry, basis, multiplicity, charge)
#molecule.load()

# Run pyscf.
molecule = ofpyscf.run_pyscf(molecule_data,
                     run_scf=1,run_mp2=1,run_cisd=1,run_ccsd=1,run_fci=1)

n_qubits    = 2*molecule.n_orbitals
n_electrons = molecule.n_electrons

ham_fermionic = of.get_fermion_operator(molecule.get_molecular_hamiltonian())
ham_jw = of.jordan_wigner(ham_fermionic)

ham_dict = qonvert.QubitOperator_to_dict(ham_jw, n_qubits)

print('Jordan-Wigner Hamiltonian:\n\n', ham_dict)

Jordan-Wigner Hamiltonian:

 {'IIII': (-0.09706626816763056+0j), 'ZIII': (0.17141282644776906+0j), 'IZII': (0.1714128264477691+0j), 'IIZI': (-0.2234315369081347+0j), 'IIIZ': (-0.2234315369081347+0j), 'ZZII': (0.16868898170361205+0j), 'YXXY': (0.045302615503799264+0j), 'YYXX': (-0.045302615503799264+0j), 'XXYY': (-0.045302615503799264+0j), 'XYYX': (0.045302615503799264+0j), 'ZIZI': (0.12062523483390414+0j), 'ZIIZ': (0.1659278503377034+0j), 'IZZI': (0.1659278503377034+0j), 'IZIZ': (0.12062523483390414+0j), 'IIZZ': (0.1744128761226159+0j)}


In order to initialise our `tapering` class, we also need to supply a reference state, along with the molecular Hamiltonian. Under the Jordan-Wigner transformation, the Hartree-Fock state for our $N$-orbital molecule with charge=0 and multiplicity=1 will be 

$$|\mathrm{HF}\rangle = |\underbrace{1 \dots 1}_{\frac{N}{2} \text{times}}\; \underbrace{0 \dots 0}_{\frac{N}{2} \text{times}} \rangle.$$

Note that OpenFermion fills orbital occupations from the left... this will not always be the case! For example, if using Qiskit or some other quantum computing package the Hartree-Fock state will not look the same.

In [3]:
hf_state = [1 for i in range(n_electrons)]+[0 for i in range(n_qubits-n_electrons)]
hf_string = ''.join([str(i) for i in hf_state])
print(f'The Hartree-Fock state is |{hf_string}>')

The Hartree-Fock state is |11111111110000>


We are now in a position to initialize our `tapering` class, which will identify a set of independent operators that generate the Hamiltonian symmetry and the symmetry sector corresponding with the reference state (in this case Hartree-Fock). The sector is obtained by measuring each symmetry generator with respect to the reference state, yielding a $\pm1$ eigenvalue.

In [4]:
taper_hamiltonian = tapering(hamiltonian=ham_dict, 
                             ref_state=hf_state)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.')
print(f'The symmetry generators are {taper_hamiltonian.symmetry_ops}')
print(f'and the sector arising from the chosen reference state is {taper_hamiltonian.symmetry_sec}.')

We are able to taper 3 qubits from the Hamiltonian.
The symmetry generators are ['ZIZIZIZIIZIZZI', 'IZIZIZIZIZIZIZ', 'IIIIIIIIZZZZII']
and the sector arising from the chosen reference state is [-1, -1, 1].


This is everything we need to go ahead and perform the tapering process, which is effected by the `taper_it()` method that calls on the parent `S3_projection` class.

In [5]:
ham_tap = taper_hamiltonian.taper_it()._dict

print('Tapered Hamiltonian:\n\n', ham_tap)

Tapered Hamiltonian:

 {'IIIIIIIIIII': -47.01998656479583, 'ZIZIZIZIZZI': -12.415916881767828, 'XZZZZZIIIZZ': -0.12899088605825926, 'XZIZIZZIZIZ': -0.12899088605825926, 'IIXZZZIIIZZ': 0.042573517206182195, 'ZIXZIZZIZIZ': 0.042573517206182195, 'IIIIXZIIIZZ': -3.561234427144212e-06, 'ZIZIXZZIZIZ': -3.561234427144212e-06, 'IIIIIIIIIXZ': 0.05262340575104057, 'ZIZIZIZIZXZ': 0.05262340575104057, 'IZIZIZZIZIZ': -12.415916881767828, 'ZXIZIZZIZIZ': 0.12899088605825937, 'ZXIIIIIIIII': 0.12899088605825937, 'ZIZXIZZIZIZ': -0.04257351720618221, 'ZZZXIIIIIII': -0.04257351720618221, 'ZIZIZXZIZIZ': 3.561234427144185e-06, 'ZZZZZXIIIII': 3.561234427144185e-06, 'ZIZIZIZIZZX': -0.05262340575104057, 'ZZZZZZIIIZX': -0.05262340575104057, 'ZIIIIIIIIII': 1.6402982909719148, 'YZYIIIIIIII': 0.12350837282018177, 'XZXIIIIIIII': 0.12350837282018177, 'YZZZYIIIIII': -1.1097999687586413e-05, 'XZZZXIIIIII': -1.1097999687586413e-05, 'YZZZZZIIIYI': 0.22229012256243133, 'XZZZZZIIIXI': 0.22229012256243133, 'IZIIIIIIIII': 1

We should also check that the ground state energy of the tapered Hamiltonian mathces that of the full system.

In [15]:
#print('Amplitude histogram for the ground state of the full system:')

true_gs_energy, true_gs_vec = exact_gs_energy(ham_dict)
#amps, true_gs = plot_ground_state_amplitudes(ham_dict, n_qubits, return_amps=True)
tap_gs_energy, tap_gs = exact_gs_energy(ham_tap)

print(f'The ground state energy of the full system is {true_gs_energy},')
print(f'whereas for the tapered system we find the energy is {tap_gs_energy}.')
print(f'The absolute error is {tap_gs_energy-true_gs_energy}.')

The ground state energy of the full system is -74.85359516972109,
whereas for the tapered system we find the energy is -74.85359516971033.
The absolute error is 1.0757617019407917e-11.


Do they match? Depending on the molecule chosen, they might not! One can sometimes find that the Hartree-Fock state does not yield the correct symmetry sector, particularly in the strongly correlated regime.

In [16]:
hf_vec = np.eye(1,len(gs),int(hf_string,2))
hf_overlap = np.square(np.abs(hf_vec.dot(true_gs_vec)))[0]

if hf_overlap < 1e-18:
    print('The Hartree-Fock state has no overlap with the true ground state!')
else:
    print('The Hartree')
print(f'Overlap of the Hartree-Fock state with the true ground state: <HF|True GS> = {hf_overlap[0]:.10f}')

Overlap of the Hartree-Fock state with the true ground state: <HF|True GS> = 0.0001936536


If we instead take the dominant basis state in the histogram above, we should see that the energies do match in the resulting sector...

In [7]:
taper_hamiltonian_2 = tapering(hamiltonian=ham_dict, 
                               ref_state=[int(i) for i in amps[0][0]])
ham_tap_2 = taper_hamiltonian_2.taper_it()._dict

tap_gs_energy_2, tap_gs_2 = exact_gs_energy(ham_tap_2)

print(f'In the sector {taper_hamiltonian_2.symmetry_sec}, we find the ground state energy to be {tap_gs_energy_2}.')
print(f'The absolute error is {tap_gs_energy_2-true_gs}.')

In the sector [1, -1, -1], we find the ground state energy to be -74.85359516971867.
The absolute error is 2.5011104298755527e-12.


*The problem is...* 

we will not in general know how the basis states are distributed in the ground state!

The scalability of tapering is highly predicated on finding new approaches to identifying the correct symmetry sector.

In [9]:
gs

array([-3.57740259e-16-2.08991716e-16j, -2.43469954e-17-5.00136382e-18j,
        1.01112395e-17-1.20306488e-17j, ...,
        2.61645427e-16+2.28332389e-16j, -2.59223831e-16+3.18776135e-16j,
       -8.34991721e-19+1.23459733e-17j])

array([3.31446024e-05])

In [12]:
molecule_data.ccsd_double_amps

array([[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e

In [19]:
ham_dict

{'IIII': (-0.09706626816763056+0j),
 'ZIII': (0.17141282644776906+0j),
 'IZII': (0.1714128264477691+0j),
 'IIZI': (-0.2234315369081347+0j),
 'IIIZ': (-0.2234315369081347+0j),
 'ZZII': (0.16868898170361205+0j),
 'YXXY': (0.045302615503799264+0j),
 'YYXX': (-0.045302615503799264+0j),
 'XXYY': (-0.045302615503799264+0j),
 'XYYX': (0.045302615503799264+0j),
 'ZIZI': (0.12062523483390414+0j),
 'ZIIZ': (0.1659278503377034+0j),
 'IZZI': (0.1659278503377034+0j),
 'IZIZ': (0.12062523483390414+0j),
 'IIZZ': (0.1744128761226159+0j)}